<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.4: Sequential Logic
**Prev: [Control Flow](2.3_control_flow.ipynb)**<br>
**Next: [FIR Filter](2.5_exercise.ipynb)**

## Motivation
You can't write any meaningful digital logic without state. You can't write any meaningful digital logic without state. You can't write any meaningful digital logic....

Get it? Because without storing intermediate results, you can't get anywhere.

Ok, that bad joke aside, this module will describe how to express common sequential patterns in Chisel. By the end of the module, you should be able to implement and test a shift register in Chisel.

It's important to emphasize that this section will probably not dramatically impress you. Chisel's power is not in new sequential logic patterns, but in the parameterization of a design. Before we demonstrate that capability, we have to learn what these sequential patterns are. Thus, this section will show you that Chisel can do pretty much what Verilog can do - you just need to learn the Chisel syntax.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/mnt/c/Users/dunn/Documents/git_working_copies/generator-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# Registers
The basic stateful element in Chisel is the register, denoted `Reg`.
A `Reg` holds its output value until the rising edge of its clock, at which time it takes on the value of its input.
By default, every Chisel `Module` has an implicit clock that is used by every register in the design.
This saves you from always specifying the same clock all over your code.

<span style="color:blue">**Example: Using a Register**</span><br>
The following code block implements a module that takes the input, adds 1 to it, and connects it as the input of a register.
*Note: The implicit clock can be overridden for multi-clock designs. See the appendix for an example.*

In [8]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val a   = Input (UInt(4.W))
    val b   = Input (UInt(2.W))
    val c   = Input (UInt(3.W))
    val out = Output(UInt())
  })
  
  val sum = Reg(UInt())
  val prod = Reg(UInt())
  sum := io.a + io.b
  prod := sum * io.c
  io.out := prod - sum
}

println(getFirrtl(new RegisterModule))
println(getVerilog(new RegisterModule))

[info] [0.000] Elaborating design...
[info] [0.070] Done elaborating.

circuit cmd7WrapperHelperRegisterModule : 
  module cmd7WrapperHelperRegisterModule : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip a : UInt<4>, flip b : UInt<2>, flip c : UInt<3>, out : UInt}
    
    reg sum : UInt, clock @[cmd7.sc 9:16]
    reg prod : UInt, clock @[cmd7.sc 10:17]
    node _T_8 = add(io.a, io.b) @[cmd7.sc 11:15]
    node _T_9 = tail(_T_8, 1) @[cmd7.sc 11:15]
    sum <= _T_9 @[cmd7.sc 11:7]
    node _T_10 = mul(sum, io.c) @[cmd7.sc 12:15]
    prod <= _T_10 @[cmd7.sc 12:8]
    node _T_11 = sub(prod, sum) @[cmd7.sc 13:18]
    node _T_12 = asUInt(_T_11) @[cmd7.sc 13:18]
    node _T_13 = tail(_T_12, 1) @[cmd7.sc 13:18]
    io.out <= _T_13 @[cmd7.sc 13:10]
    

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 1529.4 ms

module cmd7WrapperHelperRegisterModule( // @[:@3.2]
  input        clock, // @[:@4.4]
  input        reset, //

defined class RegisterModule

The register is created by calling `Reg(tpe)`, where `tpe` is a variable that encodes the type of register we want.
In this example, `tpe` is a 12-bit `UInt`.

Look at what the tester above is doing.
Between calls to `poke()` and `expect`, there is a call to `step(1)`.
This tells the test harness to tick the clock once, which will cause the register to pass its input to its output.

Calling `step(n)` will tick the clock `n` times.

The astute observer will notice that previous testers testing combinational logic did not call `step()`. This is because calling `poke()` on an input immediately propagates the updated values through combinational logic. Calling `step()` is only needed to update state elements in sequential logic. 

The code block below will show the verilog generated by `RegisterModule`.

Note:
* The module has an input for clock (and reset) that you didn't add- this is the implicit clock
* The variable `register` shows up as `reg [11:0]`, as expected
* There is a block sectioned off by ` `ifdef Randomize` that initialized the register to some random variable before simulation starts
* `register` is updated on `posedge clock`

In [4]:
println(getVerilog(new RegisterModule))

[info] [0.001] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 157.5 ms

module cmd2WrapperHelperRegisterModule( // @[:@3.2]
  input         clock, // @[:@4.4]
  input         reset, // @[:@5.4]
  input  [11:0] io_in, // @[:@6.4]
  output [11:0] io_out // @[:@6.4]
);
  reg [11:0] register; // @[cmd2.sc 7:21:@8.4]
  reg [31:0] _RAND_0;
  wire [12:0] _T_6; // @[cmd2.sc 8:21:@9.4]
  wire [11:0] _T_7; // @[cmd2.sc 8:21:@10.4]
  assign _T_6 = io_in + 12'h1; // @[cmd2.sc 8:21:@9.4]
  assign _T_7 = _T_6[11:0]; // @[cmd2.sc 8:21:@10.4]
  assign io_out = register;
`ifdef RANDOMIZE
  integer initvar;
  initial begin
    `ifndef verilator
      #0.002 begin end
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{$random}};
  register = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  end
`endif // RANDOMIZE
  always @(posedge clock) begin
    register <= _T_7;
  end
endmodule



One important note is that Chisel distinguishes between types (like `UInt`) and hardware nodes (like the literal `2.U`, or the output of `myReg`). While
```scala
val myReg = Reg(UInt(2.W))
```
is legal because a Reg needs a data type as a model,
```scala
val myReg = Reg(2.U)
```
is an error because `2.U` is already a hardware node and can't be used as a model.

<span style="color:blue">**Example: RegNext**</span><br>
Chisel has a convenience register object for registers with simple input connections. The previous `Module` can be shortened to the following `Module`. Notice how we didn't need to specify the register bitwidth this time. It gets inferred from the register's output connection, in this case `io.out`.

In [5]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input (UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

class RegNextModuleTester(c: RegNextModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegNextModule) { c => new RegNextModuleTester(c) })
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 41.4 ms
Total FIRRTL Compile Time: 15.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1536081558494
test cmd4WrapperHelperRegNextModule Success: 100 tests passed in 105 cycles taking 0.149247 seconds
[info] [0.147] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegNextModule
defined class RegNextModuleTester

The Verilog looks almost the same as before, though the register name is generated instead of explicity defined.

In [6]:
println(getVerilog(new RegNextModule))

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 22.6 ms

module cmd4WrapperHelperRegNextModule( // @[:@3.2]
  input         clock, // @[:@4.4]
  input         reset, // @[:@5.4]
  input  [11:0] io_in, // @[:@6.4]
  output [11:0] io_out // @[:@6.4]
);
  wire [12:0] _T_5; // @[cmd4.sc 8:27:@8.4]
  wire [11:0] _T_6; // @[cmd4.sc 8:27:@9.4]
  reg [11:0] _T_8; // @[cmd4.sc 8:20:@10.4]
  reg [31:0] _RAND_0;
  assign _T_5 = io_in + 12'h1; // @[cmd4.sc 8:27:@8.4]
  assign _T_6 = _T_5[11:0]; // @[cmd4.sc 8:27:@9.4]
  assign io_out = _T_8;
`ifdef RANDOMIZE
  integer initvar;
  initial begin
    `ifndef verilator
      #0.002 begin end
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{$random}};
  _T_8 = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  end
`endif // RANDOMIZE
  always @(posedge clock) begin
    _T_8 <= _T_6;
  end
endmodule



---
# `RegInit`

The register in `RegisterModule` was initialized to random data for simulation.
Unless otherwised specified, registers do not have a reset value (or a reset).
The way to create a register that resets to a given value is with `RegInit`.

For instance, a 12-bit register initialized to zero can be created with the following.
Both versions below are valid and do the same thing:
```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```

The first version has two arguments.
The first argument is a type node that specified the datatype and its width.
The second argument is a hardware node that specified the reset value, in this case 0.

The second version has one argument.
It is a hardware node that specifies the reset value, but normally `0.U`.

<span style="color:blue">**Example: Initialized Register** </span><br>
The following demonstrates using `RegInit()`, initialized to zero.

In [7]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input (UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

println(getVerilog(new RegInitModule))

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 20.3 ms

module cmd6WrapperHelperRegInitModule( // @[:@3.2]
  input         clock, // @[:@4.4]
  input         reset, // @[:@5.4]
  input  [11:0] io_in, // @[:@6.4]
  output [11:0] io_out // @[:@6.4]
);
  reg [11:0] register; // @[cmd6.sc 7:25:@8.4]
  reg [31:0] _RAND_0;
  wire [12:0] _T_7; // @[cmd6.sc 8:21:@9.4]
  wire [11:0] _T_8; // @[cmd6.sc 8:21:@10.4]
  assign _T_7 = io_in + 12'h1; // @[cmd6.sc 8:21:@9.4]
  assign _T_8 = _T_7[11:0]; // @[cmd6.sc 8:21:@10.4]
  assign io_out = register;
`ifdef RANDOMIZE
  integer initvar;
  initial begin
    `ifndef verilator
      #0.002 begin end
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{$random}};
  register = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  end
`endif // RANDOMIZE
  always @(posedge clock) begin
    if (reset) begin
      register <= 12'h0;
    end else begin
      register <= _T_8;
    end
  end
endmodule



defined class RegInitModule

Note that the generated verilog now has a block that checks `if (reset)` to reset the register to 0.
Also note that this is inside the `always @(posedge clock)` block.
Chisel's implicit reset is active high and synchronous.
The register is still initialized to random junk before reset is called.
The `PeekPokeTesters` always call reset before running your test, but you can manually call reset as well using the `reset(n)` function, where reset is high for `n` cycles.

---
# Control Flow
Registers are very similar to wires in terms of control flow.
They have last connect semantics and can be assigned to conditionally with `when`, `elsewhen`, and `otherwise`.

<span style="color:blue">**Example: Register Control Flow**</span><br>
The following example finds the maximum value in a sequence of inputs using conditional register assignments.

In [8]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}
assert(chisel3.iotesters.Driver(() => new FindMax) {
  c => new PeekPokeTester(c) {
    expect(c.io.max, 0)
    poke(c.io.in, 1)
    step(1)
    expect(c.io.max, 1)
    poke(c.io.in, 3)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 2)
    step(1)
    expect(c.io.max, 3)
    poke(c.io.in, 24)
    step(1)
    expect(c.io.max, 24)
  }
})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 21.9 ms
Total FIRRTL Compile Time: 19.0 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1536081843926
test cmd7WrapperHelperFindMax Success: 5 tests passed in 9 cycles taking 0.008140 seconds
[info] [0.006] RAN 4 CYCLES PASSED
SUCCESS!!


defined class FindMax

---
# Other Register Examples

Operations called on a register are performed on the **output** of the register, and the kind of operations depend on the register's type.
That means that you can write
```scala
val reg: UInt = Reg(UInt(4.W))```
which means the value `reg` is of type `UInt` and you can do things you can normally do with `UInt`s, like `+`, `-`, etc.


You aren't restricted to using `UInt`s with registers, you can use any subclass of the base type `chisel3.Data`. This includes `SInt` for signed integers and a lot of other things.

<span style="color:blue">**Example: Comb Filter**</span><br>
The following example shows a comb filter.

In [9]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

[info] [0.000] Elaborating design...
[info] [0.009] Done elaborating.
Total FIRRTL Compile Time: 27.1 ms

module cmd8WrapperHelperComb( // @[:@3.2]
  input         clock, // @[:@4.4]
  input         reset, // @[:@5.4]
  input  [11:0] io_in, // @[:@6.4]
  output [11:0] io_out // @[:@6.4]
);
  reg [11:0] delay; // @[cmd8.sc 7:24:@8.4]
  reg [31:0] _RAND_0;
  wire [12:0] _T_5; // @[cmd8.sc 9:19:@10.4]
  wire [11:0] _T_6; // @[cmd8.sc 9:19:@11.4]
  wire [11:0] _T_7; // @[cmd8.sc 9:19:@12.4]
  assign _T_5 = $signed(io_in) - $signed(delay); // @[cmd8.sc 9:19:@10.4]
  assign _T_6 = _T_5[11:0]; // @[cmd8.sc 9:19:@11.4]
  assign _T_7 = $signed(_T_6); // @[cmd8.sc 9:19:@12.4]
  assign io_out = _T_7;
`ifdef RANDOMIZE
  integer initvar;
  initial begin
    `ifndef verilator
      #0.002 begin end
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{$random}};
  delay = _RAND_0[11:0];
  `endif // RANDOMIZE_REG_INIT
  end
`endif // RANDOMIZE
  always @(posedge clock) begin
    delay <= io_in;
  end

defined class Comb

---
# Exercises
<span style="color:red">**Exercise: Shift Register**</span><br>
Given your new-found registering knowledge, build a module that implements a shift register for a LFSR. Specifically:
- Each element is a single bit wide.
- Has a 4-bit output signal.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- **The output initializes at `b0001`.**
- Shifts each clock cycle (no enable signal).
- **Note in Chisel, subword assignment IS ILLEGAL**; something like `out(0) := in` will not work.

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [20]:
class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)
  val next_state = Wire(UInt(4.W))
  next_state := Cat(state(2), state(1), state(0), io.in)
  state := next_state
  io.out := state
  
}

class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    poke(c.io.in, i % 2)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    step(1)
    expect(c.io.out, state)
  }
}
assert(chisel3.iotesters.Driver(() => new MyShiftRegister()) {
    c => new MyShiftRegisterTester(c)
})
println("SUCCESS!!")

[info] [0.000] Elaborating design...
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 18.0 ms
Total FIRRTL Compile Time: 16.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1536083837322
test cmd19WrapperHelperMyShiftRegister Success: 10 tests passed in 15 cycles taking 0.007899 seconds
[info] [0.007] RAN 10 CYCLES PASSED
SUCCESS!!


defined class MyShiftRegister
defined class MyShiftRegisterTester

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Shift Register (Optional)**</span><br>
Write a shift register that is parameterized by its delay (`n`), its initial value (`init`), and also has an enable input signal (`en`).

In [9]:
// n is the output width (number of delays - 1)
// init state to init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))
  val next_state = Wire(UInt(n.W))
  // Can just use bit shifting and bitwise OR
  next_state := ((state << 1) | io.in)
  when (io.en) {state := next_state}
  // I think io.out should be assigned to state, as in the previous exercise.
  io.out := next_state

}

class MyOptionalShiftRegisterTester(c: MyOptionalShiftRegister) extends PeekPokeTester(c) {
  val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
  var state = c.init
  var i = 0
  poke(c.io.en, 1)
  while (i < 10 * c.n) {
    // poke in repeated inSeq
    val toPoke = inSeq(i % inSeq.length)
    poke(c.io.in, toPoke)
    // update expected state
    state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
    expect(c.io.out, state)
    step(1)

    i += 1
  }
}

// test different depths
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  assert(chisel3.iotesters.Driver(() => new MyOptionalShiftRegister(n = i)) {
    c => new MyOptionalShiftRegisterTester(c)
  })
}
println("SUCCESS!!")

Testing n=3
[info] [0.000] Elaborating design...
[info] [0.005] Done elaborating.
Total FIRRTL Compile Time: 14.2 ms
Total FIRRTL Compile Time: 15.0 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1536110690413
test cmd8WrapperHelperMyOptionalShiftRegister Success: 30 tests passed in 35 cycles taking 0.010246 seconds
[info] [0.007] RAN 30 CYCLES PASSED
Testing n=4
[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 15.4 ms
Total FIRRTL Compile Time: 11.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1536110690483
test cmd8WrapperHelperMyOptionalShiftRegister Success: 40 tests passed in 45 cycles taking 0.005827 seconds
[info] [0.006] RAN 40 CYCLES PASSED
Testing n=8
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 15.6 ms
Total FIRRTL Compile Time: 18.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1536110690540
test cmd8Wrapper

defined class MyOptionalShiftRegister
defined class MyOptionalShiftRegisterTester

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := nextState
</pre></article></div></section></div>

---
# Appendix: Explicit clock and reset
Chisel modules have a default clock and reset that are implicitly used by every register created inside them.
There are times where you want to be able to override this default behavior; perhaps you have a black box that generates a clock or reset signal, or you have a multi-clock design.

Chisel provides constructs for dealing with these cases.
Clocks and resets can be overridden separately or together with `withClock() {}`, `withReset() {}`, and `withClockAndReset() {}`.
The following code blocks will give examples of using these functions.

One thing to be aware of is that `reset` (as of this tutorial's writing) is always synchronous and of type `Bool`.
Clocks have their own type in Chisel (`Clock`) and should be declared as such.
*`Bool`s can be converted to `Clock`s by calling `asClock()` on them, but you should be careful that you aren't doing something silly.*

Also note that `chisel-testers` do not currently have complete support for multi-clock designs.

<span style="color:blue">**Example: Multi-Clock Module**</span><br>
A module with multiple clocks and reset signals.

In [11]:
// we need to import multi-clock features
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    // everything in this scope with have alternateReset as the reset
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

[info] [0.000] Elaborating design...
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 19.3 ms

module cmd10WrapperHelperClockExamples( // @[:@3.2]
  input        clock, // @[:@4.4]
  input        reset, // @[:@5.4]
  input  [9:0] io_in, // @[:@6.4]
  input        io_alternateReset, // @[:@6.4]
  input        io_alternateClock, // @[:@6.4]
  output [9:0] io_outImplicit, // @[:@6.4]
  output [9:0] io_outAlternateReset, // @[:@6.4]
  output [9:0] io_outAlternateClock, // @[:@6.4]
  output [9:0] io_outAlternateBoth // @[:@6.4]
);
  reg [9:0] imp; // @[cmd10.sc 14:20:@8.4]
  reg [31:0] _RAND_0;
  reg [9:0] _T_13; // @[cmd10.sc 20:25:@11.4]
  reg [31:0] _RAND_1;
  reg [9:0] _T_16; // @[cmd10.sc 26:25:@14.4]
  reg [31:0] _RAND_2;
  reg [9:0] _T_19; // @[cmd10.sc 32:22:@17.4]
  reg [31:0] _RAND_3;
  assign io_outImplicit = imp;
  assign io_outAlternateReset = _T_13;
  assign io_outAlternateClock = _T_16;
  assign io_outAlternateBoth = _T_19;
`ifdef RANDOMIZE
  integer initvar;
  ini

import chisel3.experimental.{withClock, withReset, withClockAndReset}


defined class ClockExamples

---
# Wrap Up
Great job completing this section!! You've now learned how to create registers and write sequential logic in Chisel, which means you have enough basic building blocks to write real circuits.

The next section will combine everything we've learned into one example! If you need a little more encouragement, just remember these words from an expert Chisel user:

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# You're done!

[Return to the top.](#top)